In [1]:
import mlflow
import mlflow.sklearn
import dagshub
import pandas as pd
import numpy as np
import missingno as msn
from utility import basic_data_cleaning
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer,MissingIndicator
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from EDA import impute_analysis
from sklearn.model_selection import train_test_split, cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"E:\DATA SCIENCE & AI\PROJECTS\Swiggy Delivery Dataset\train.csv")

In [3]:
df.sample(6)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
15660,0x413a,BANGRES04DEL01,38,4.9,12.980410,77.640489,13.020410,77.680489,11-03-2022,15:30:00,15:35:00,conditions Windy,Medium,2,Meal,motorcycle,1,No,Urban,(min) 28
7586,0xc1b7,KOCRES04DEL02,26,4.5,9.966783,76.242981,10.046783,76.322981,16-02-2022,21:30:00,21:40:00,conditions Windy,Jam,2,Buffet,electric_scooter,1,No,Metropolitian,(min) 21
18938,0x6a68,INDORES05DEL01,26,5,22.727021,75.884167,22.767021,75.924167,21-03-2022,14:35:00,14:45:00,conditions Stormy,High,1,Snack,motorcycle,0,No,Metropolitian,(min) 15
24571,0x96f,COIMBRES18DEL02,NaN,NaN,11.020910,76.940432,11.150910,77.070432,29-03-2022,20:55:00,21:10:00,conditions Sunny,Jam,1,Snack,scooter,1,No,Urban,(min) 30
29013,0x8a1,MUMRES01DEL02,NaN,NaN,19.126630,72.829976,19.206630,72.909976,20-03-2022,NaN,17:50:00,conditions Stormy,Medium,0,Buffet,motorcycle,1,No,Urban,(min) 47
9786,0xcea7,AGRRES17DEL02,25,5,27.157772,78.047250,27.207772,78.097250,11-02-2022,17:15:00,17:25:00,conditions Cloudy,Medium,2,Snack,electric_scooter,0,No,Metropolitian,(min) 12


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [5]:
df = basic_data_cleaning(df)

In [6]:
df.sample(5)

,Delivery_person_Age,Delivery_person_Ratings,Distance_res_loc_KM,order_dayname,time_of_day,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
8099,21.0,4.8,16.385071,Sunday,night,cloudy,low,1,motorcycle,0.0,no,urban,24.0
6501,29.0,4.6,13.829790,Friday,night,windy,jam,2,scooter,1.0,no,metropolitian,18.0
21832,35.0,4.0,4.560981,Thursday,evening,sandstorms,jam,2,scooter,1.0,no,metropolitian,31.0
29312,23.0,4.9,19.761077,Saturday,night,cloudy,jam,1,motorcycle,1.0,no,metropolitian,37.0
6808,35.0,4.5,3.116147,Sunday,morning,sunny,high,0,motorcycle,0.0,no,NaN,20.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45502 entries, 0 to 45592
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Delivery_person_Age      43648 non-null  float64 
 1   Delivery_person_Ratings  43594 non-null  float64 
 2   Distance_res_loc_KM      41872 non-null  float64 
 3   order_dayname            45502 non-null  object  
 4   time_of_day              43862 non-null  category
 5   Weatherconditions        44977 non-null  object  
 6   Road_traffic_density     44992 non-null  object  
 7   Vehicle_condition        45502 non-null  int64   
 8   Type_of_vehicle          45502 non-null  object  
 9   multiple_deliveries      44509 non-null  float64 
 10  Festival                 45274 non-null  object  
 11  City                     44304 non-null  object  
 12  Time_taken(min)          45502 non-null  float64 
dtypes: category(1), float64(5), int64(1), object(6)
memory usage: 4.6+

In [8]:
df.isnull().sum()

Delivery_person_Age        1854
Delivery_person_Ratings    1908
Distance_res_loc_KM        3630
order_dayname                 0
time_of_day                1640
Weatherconditions           525
Road_traffic_density        510
Vehicle_condition             0
Type_of_vehicle               0
multiple_deliveries         993
Festival                    228
City                       1198
Time_taken(min)               0
dtype: int64

In [9]:
round(df.isnull().sum()/df.shape[0]*100,2)

Delivery_person_Age        4.07
Delivery_person_Ratings    4.19
Distance_res_loc_KM        7.98
order_dayname              0.00
time_of_day                3.60
Weatherconditions          1.15
Road_traffic_density       1.12
Vehicle_condition          0.00
Type_of_vehicle            0.00
multiple_deliveries        2.18
Festival                   0.50
City                       2.63
Time_taken(min)            0.00
dtype: float64

In [10]:
df.shape[0] - df.isnull().sum().sum()

np.int64(33016)

In [11]:
df.isnull().sum().sum()/df.shape[0]

np.float64(0.2744055206364555)

_______________________________________

### **Experimenting with Imputation:**

In [12]:
num_cols = ['Delivery_person_Age','Delivery_person_Ratings','Distance_res_loc_KM','Vehicle_condition','multiple_deliveries']

adv_num_cols = ['Delivery_person_Age','Delivery_person_Ratings','Distance_res_loc_KM']
cont_num_cols = ['multiple_deliveries']

#===============================================================================================================
cat_cols = ['order_dayname','time_of_day','Weatherconditions','Road_traffic_density','Type_of_vehicle','Festival','City']
mode_cat_cols = ['time_of_day','Road_traffic_density','Type_of_vehicle','Weatherconditions','Festival','City']

In [13]:
trafic_list = ['low','medium','high','jam']

In [14]:
scale = StandardScaler()

In [15]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
X_train

,Delivery_person_Age,Delivery_person_Ratings,Distance_res_loc_KM,order_dayname,time_of_day,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City
11047,26.0,4.7,9.177697,Wednesday,night,windy,jam,2,electric_scooter,1.0,no,metropolitian
15692,21.0,4.9,6.081544,Thursday,afternoon,windy,medium,2,motorcycle,1.0,no,metropolitian
14499,NaN,NaN,1.554565,Saturday,NaN,stormy,low,2,scooter,1.0,no,metropolitian
29258,32.0,4.7,20.852300,Monday,evening,windy,medium,0,motorcycle,1.0,no,metropolitian
22554,34.0,4.9,9.056108,Wednesday,night,sandstorms,jam,1,scooter,0.0,no,urban
...,...,...,...,...,...,...,...,...,...,...,...,...
11303,29.0,4.7,12.464385,Monday,evening,sandstorms,jam,2,scooter,0.0,no,metropolitian
44821,36.0,4.8,1.520434,Wednesday,morning,cloudy,low,0,motorcycle,0.0,no,urban
38234,36.0,4.4,NaN,Thursday,evening,fog,jam,1,scooter,0.0,no,metropolitian
860,25.0,4.9,7.760948,Saturday,evening,windy,medium,2,scooter,1.0,no,metropolitian


In [18]:
mode_nom_cat_cols = ['order_dayname','time_of_day','Type_of_vehicle','Weatherconditions','Festival','City']
mode_ord_cat_cols = ['Road_traffic_density']

In [19]:
trasformer1 = ColumnTransformer(
    transformers=[
        ("KNN_imputer",KNNImputer(),adv_num_cols),
        ("Constant_imputer_num",SimpleImputer(strategy='constant',fill_value=0.0),cont_num_cols),
        ("mode_imp_nominal_encode_cat",
         Pipeline(
             steps=[
                 ("Mode_imp_cat",SimpleImputer(strategy='most_frequent')),
                 ("OHE",OneHotEncoder(drop='first',sparse_output=False))
             ]
         ),mode_nom_cat_cols),
         ("mode_ord_imp_cats",
          Pipeline(
              steps=[
                  ("mode_imp",SimpleImputer(strategy='most_frequent')),
                  ("Ordinal",OrdinalEncoder(categories=[trafic_list],handle_unknown='use_encoded_value',unknown_value=-1,encoded_missing_value=-2))
              ]
          ),mode_ord_cat_cols)
    ],remainder='passthrough'
)

In [20]:
processor = Pipeline(
    steps=[
        ("First_transformation",trasformer1),
        ("scaling",StandardScaler())
    ]
)
processor.set_output(transform=None)

,steps,"[('First_transformation', ...), ('scaling', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('KNN_imputer', ...), ('Constant_imputer_num', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [21]:
X_train

,Delivery_person_Age,Delivery_person_Ratings,Distance_res_loc_KM,order_dayname,time_of_day,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City
11047,26.0,4.7,9.177697,Wednesday,night,windy,jam,2,electric_scooter,1.0,no,metropolitian
15692,21.0,4.9,6.081544,Thursday,afternoon,windy,medium,2,motorcycle,1.0,no,metropolitian
14499,NaN,NaN,1.554565,Saturday,NaN,stormy,low,2,scooter,1.0,no,metropolitian
29258,32.0,4.7,20.852300,Monday,evening,windy,medium,0,motorcycle,1.0,no,metropolitian
22554,34.0,4.9,9.056108,Wednesday,night,sandstorms,jam,1,scooter,0.0,no,urban
...,...,...,...,...,...,...,...,...,...,...,...,...
11303,29.0,4.7,12.464385,Monday,evening,sandstorms,jam,2,scooter,0.0,no,metropolitian
44821,36.0,4.8,1.520434,Wednesday,morning,cloudy,low,0,motorcycle,0.0,no,urban
38234,36.0,4.4,NaN,Thursday,evening,fog,jam,1,scooter,0.0,no,metropolitian
860,25.0,4.9,7.760948,Saturday,evening,windy,medium,2,scooter,1.0,no,metropolitian


In [22]:
X_train_transformed = processor.fit_transform(X_train)

In [23]:
X_train_transformed

array([[-0.63226772,  0.2098912 , -0.11138343, ..., -0.53320072,
         1.30686302,  1.17593682],
       [-1.51611323,  0.85750055, -0.68057577, ..., -0.53320072,
        -0.29718057,  1.17593682],
       [-0.17266805,  0.08036933, -1.51280945, ..., -0.53320072,
        -1.09920237,  1.17593682],
       ...,
       [ 1.13542331, -0.76152283,  0.32711949, ..., -0.53320072,
         1.30686302, -0.02238563],
       [-0.80903682,  0.85750055, -0.37183643, ..., -0.53320072,
        -0.29718057,  1.17593682],
       [-0.10196041,  0.53369587, -0.37944686, ..., -0.53320072,
         1.30686302,  1.17593682]], shape=(36401, 27))

In [24]:
X_test_transformed = processor.transform(X_test)

In [25]:
dagshub.init(repo_owner='Pravat-21', repo_name='Swiggy-Delivery-Time-Prediction', mlflow=True)

Accessing as Pravat-21

Initialized MLflow to track repo "Pravat-21/Swiggy-Delivery-Time-Prediction"

Repository Pravat-21/Swiggy-Delivery-Time-Prediction initialized!

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error

In [27]:
mlflow.set_experiment("Exp 1-Baseline Model with Imputation")

<Experiment: artifact_location='mlflow-artifacts:/5a83599b695b42ba8c5dbbfd6a5b8a95', creation_time=1764874127756, experiment_id='0', last_update_time=1764874127756, lifecycle_stage='active', name='Exp 1-Baseline Model with Imputation', tags={}>

In [31]:
def create_processor(imputer,scale,imputation):

    if imputation == True:

        trasformer1 = ColumnTransformer(
        transformers=[
            ("Adv_imputer",imputer,adv_num_cols),
            ("Constant_imputer_num",SimpleImputer(strategy='constant',fill_value=0.0),cont_num_cols),
            ("mode_imp_nominal_encode_cat",
            Pipeline(
                steps=[
                    ("Mode_imp_cat",SimpleImputer(strategy='most_frequent')),
                    ("OHE",OneHotEncoder(drop='first',sparse_output=False))
                ]
            ),mode_nom_cat_cols),
            ("mode_ord_imp_cats",
            Pipeline(
                steps=[
                    ("mode_imp",SimpleImputer(strategy='most_frequent')),
                    ("Ordinal",OrdinalEncoder(categories=[trafic_list],handle_unknown='use_encoded_value',unknown_value=-1,encoded_missing_value=-2))
                ]
            ),mode_ord_cat_cols)
        ],remainder='passthrough'
        )
    else:
        trasformer1 = ColumnTransformer(
            transformers=[
                ("OHE",OneHotEncoder(drop='first',sparse_output=False),mode_nom_cat_cols),
                ("Ordinal",OrdinalEncoder(categories=[trafic_list],handle_unknown='use_encoded_value',unknown_value=-1,encoded_missing_value=-2),mode_ord_cat_cols)
            ],remainder="passthrough"
        )

    processor = Pipeline(
    steps=[
        ("First_transformation",trasformer1),
        ("scaling",scale)
    ]
    )

    return processor

In [28]:
pt = PowerTransformer()

In [78]:
def log_mlflow(scaling,imputing,y_transform = False,imputation=True):
    if imputation == False:
        imputing = 'No'

    with mlflow.start_run(run_name=f"(RF)-Imputation({imputing})-({scaling}scaler)-y_trns({y_transform})") as run:

        mlflow.set_tags({
            "model":"Random Forest Regressor",
            "experiment_type":"baseline-imputation",
            "Description":f"Here with {imputing}Imputer is used for advanced imputation."
        })
        if imputing == 'KNN':
            imputer = KNNImputer()
        elif imputing == 'iterative':
            imputer = IterativeImputer()
        else:
            imputer = None

        if scaling == 'standard':
            scale = StandardScaler()
        else:
            scale = MinMaxScaler()

        processor = create_processor(imputer,scale,imputation)

        if imputation == True:

            X_train_transformed = processor.fit_transform(X_train)
            X_test_transformed = processor.transform(X_test)
        else:
            copy_df = df.dropna()
            X = copy_df.iloc[:,:-1]
            y=copy_df.iloc[:,-1]
            X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
            X_train_transformed = processor.fit_transform(X_train)
            X_test_transformed = processor.transform(X_test)


        if y_transform == True:
            model = TransformedTargetRegressor(regressor=RandomForestRegressor(),transformer=pt)
        else:
            model = RandomForestRegressor()

        model.fit(X_train_transformed,y_train)

        y_pred_train = model.predict(X_train_transformed)
        y_pred = model.predict(X_test_transformed)

        r2score_test = r2_score(y_test,y_pred)
        r2score_train = r2_score(y_train,y_pred_train)

        mae = mean_absolute_error(y_test,y_pred)
        mae_train = mean_absolute_error(y_train,y_pred_train)

        rmse = root_mean_squared_error(y_test,y_pred)
        rmse_train = root_mean_squared_error(y_train,y_pred_train)

        mlflow.log_metrics({
            "training_R2_score":r2score_train,
            "testing_R2_score":r2score_test,
            "training_MAE":mae_train,
            "testing_mae":mae,
            "training_rmse":rmse_train,
            "testing_rmse":rmse
        })
        
        mlflow.log_params({
            "model":"RandomForestRegressor",
            "advance_Imputer":f"{imputing}Imputer",
            "y-Trasformation":y_transform,
            "scaler": f"{scaling}"

        })

        print({
            "training_R2_score":r2score_train,
            "testing_R2_score":r2score_test,
            "training_MAE":mae_train,
            "testing_mae":mae,
            "training_rmse":rmse_train,
            "testing_rmse":rmse
        })

In [66]:
scaling = ['standard','minmax']
imputing = ['KNN','iterative']
y_transform = [True,False]

In [60]:
for i in scaling:
    for j in imputing:
        for k in y_transform:
                log_mlflow(scaling=i,imputing=j,y_transform=k,imputation=True)

{'training_R2_score': 0.9701965366411468, 'testing_R2_score': 0.7894718695066227, 'training_MAE': 1.2386994699188285, 'testing_mae': 3.35408594100985, 'training_rmse': 1.6207224833048854, 'testing_rmse': 4.303285839773535}
🏃 View run (RF)-Imputation(KNN)-(standardscaler)-y_trns(True) at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0/runs/df0a3de0b0f840608cebb5b5e0ff07d6
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0
{'training_R2_score': 0.9709634475660476, 'testing_R2_score': 0.7923115943689604, 'training_MAE': 1.2316882097692239, 'testing_mae': 3.343991719905191, 'training_rmse': 1.599734145348517, 'testing_rmse': 4.274164704879567}
🏃 View run (RF)-Imputation(KNN)-(standardscaler)-y_trns(False) at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0/runs/87cfcab3cf434815841a6a4d2a37927a
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-

In [67]:
scaling = ['standard','minmax']
y_transform = [True,False]

In [79]:
for i in scaling:
    for k in y_transform:
            log_mlflow(scaling=i,imputing='no',y_transform=k,imputation=False)

{'training_R2_score': 0.9755600093337822, 'testing_R2_score': 0.8220832763930015, 'training_MAE': 1.149759479320084, 'testing_mae': 3.1526894597997264, 'training_rmse': 1.4577910463722341, 'testing_rmse': 3.944597266795234}
🏃 View run (RF)-Imputation(No)-(standardscaler)-y_trns(True) at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0/runs/52920200d2cb4d8482a7f09674bedff6
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0
{'training_R2_score': 0.9758679673929767, 'testing_R2_score': 0.8197226943100042, 'training_MAE': 1.1467649666677613, 'testing_mae': 3.1743451990017078, 'training_rmse': 1.4485774237140971, 'testing_rmse': 3.970679308112246}
🏃 View run (RF)-Imputation(No)-(standardscaler)-y_trns(False) at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/0/runs/8442fa1f54f741a5bff5f764b7bc650e
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery

- **After comapring all these scores It is decided to drop all the `NaN` values. As it gives us the best result.**

- **Also we will use `StandardScaler`.** 
 
- **y_transform is set to be `True`.**


________________________________________